## Множественная проверка гипотез

## Task 1
Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле: AUCs.txt

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.




In [6]:
import numpy as np
import pandas as pd
from scipy.stats import wilcoxon
from itertools import combinations

In [2]:
data = pd.read_csv('datasets/AUCs.txt', sep='\t')
data.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [12]:
records = []
for cls1, cls2 in combinations(data.columns[1:], 2):
    wilcoxon_score = wilcoxon(data[cls1], data[cls2])
    records.append((cls1, cls2, wilcoxon_score.pvalue, wilcoxon_score.statistic))

In [13]:
data_comparison = pd.DataFrame.from_records(records, columns=['cls1', 'cls2', 'p_value', 'statistic'])
data_comparison.head()

,cls1,cls2,p_value,statistic
0,C4.5,C4.5+m,0.010757,6.5
1,C4.5,C4.5+cf,0.861262,43.0
2,C4.5,C4.5+m+cf,0.015906,11.0
3,C4.5+m,C4.5+cf,0.046333,17.0
4,C4.5+m,C4.5+m+cf,0.327826,22.0


In [14]:
data_comparison.sort_values(by='p_value')

,cls1,cls2,p_value,statistic
0,C4.5,C4.5+m,0.010757,6.5
2,C4.5,C4.5+m+cf,0.015906,11.0
5,C4.5+cf,C4.5+m+cf,0.022909,10.0
3,C4.5+m,C4.5+cf,0.046333,17.0
4,C4.5+m,C4.5+m+cf,0.327826,22.0
1,C4.5,C4.5+cf,0.861262,43.0


**Ответ:** C4.5 и C4.5+m

Сколько статистически значимых на уровне 0.05 различий мы обнаружили? **4**  
Настройка какого из параметров классификатора даёт более значимое увеличение качества? **m**

## Task 2
Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [11]:
from statsmodels.sandbox.stats.multicomp import 

In [15]:
multipletests(data_comparison.p_value, method='holm', alpha=0.05)

(array([False, False, False, False, False, False]),
 array([0.0645428 , 0.86126233, 0.07953222, 0.13899819, 0.65565135,
        0.0916364 ]),
 0.008512444610847103,
 0.008333333333333333)

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [16]:
multipletests(data_comparison.p_value, method='fdr_bh', alpha=0.05)

(array([ True, False,  True, False, False,  True]),
 array([0.0458182 , 0.86126233, 0.0458182 , 0.06949909, 0.39339081,
        0.0458182 ]),
 0.008512444610847103,
 0.008333333333333333)

 Что можно сделать, чтобы увеличить вероятность обнаружения различий, если они действительно существуют?  
 **Ответ:** взять больше датасетов